In [1]:
#load Libraries
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams["figure.figsize"]=(20,10)

In [2]:
#read csv file
df1=pd.read_csv("Bengaluru_House_Data.csv")
df1.head()


,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


In [3]:
#check data's shape
df1.shape

#data has 9 columns and 13,320 entries

(13320, 9)

In [4]:
#count of each area type sample
df1.groupby('area_type')['area_type'].agg('count')

area_type
Built-up  Area          2418
Carpet  Area              87
Plot  Area              2025
Super built-up  Area    8790
Name: area_type, dtype: int64

In [5]:
#drop areas that we dont need to make our model. eg house availability,society etc.
df2 = df1.drop(["area_type", "society", "balcony", "availability"], axis=1)
print(df2.shape)



(13320, 5)


In [22]:
#preview our new dataset
df2.head()

,location,size,total_sqft,bath,price
0,Electronic City Phase II,2 BHK,1056,2.0,39.07
1,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00
2,Uttarahalli,3 BHK,1440,2.0,62.00
3,Lingadheeranahalli,3 BHK,1521,3.0,95.00
4,Kothanur,2 BHK,1200,2.0,51.00


In [ ]:
#Data Cleaning

In [6]:
#check for null values in the data
df2.isnull().sum()

location       1
size          16
total_sqft     0
bath          73
price          0
dtype: int64

In [7]:
#drop the null rows, as they are of less significance to teh dataset
df3=df2.dropna()

#checking for null values again
df3.isnull().sum()

location      0
size          0
total_sqft    0
bath          0
price         0
dtype: int64

In [8]:
#check the shape of new dataset
df3.shape

(13246, 5)

In [26]:
#data exploration, notably on the size feature(column)
df3['size'].unique()

#high likely, the '4 Bedroom' and '4 BHK' and many other similar entries refer to same thing

array(['2 BHK', '4 Bedroom', '3 BHK', '4 BHK', '6 Bedroom', '3 Bedroom',
       '1 BHK', '1 RK', '1 Bedroom', '8 Bedroom', '2 Bedroom',
       '7 Bedroom', '5 BHK', '7 BHK', '6 BHK', '5 Bedroom', '11 BHK',
       '9 BHK', '9 Bedroom', '27 BHK', '10 Bedroom', '11 Bedroom',
       '10 BHK', '19 BHK', '16 BHK', '43 Bedroom', '14 BHK', '8 BHK',
       '12 Bedroom', '13 BHK', '18 Bedroom'], dtype=object)

In [9]:
#work around the size the column to ensure uniformity
df3['bhk']=df3['size'].apply(lambda x: int(x.split(' ')[0]))

C:\Users\HP ENVY\AppData\Local\Temp\ipykernel_11708\2712108196.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['bhk']=df3['size'].apply(lambda x: int(x.split(' ')[0]))


In [10]:
#check our new df
df3.head()

,location,size,total_sqft,bath,price,bhk
0,Electronic City Phase II,2 BHK,1056,2.0,39.07,2
1,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00,4
2,Uttarahalli,3 BHK,1440,2.0,62.00,3
3,Lingadheeranahalli,3 BHK,1521,3.0,95.00,3
4,Kothanur,2 BHK,1200,2.0,51.00,2


In [11]:
#check the number of bedrooms per house
df3['bhk'].unique()

array([ 2,  4,  3,  6,  1,  8,  7,  5, 11,  9, 27, 10, 19, 16, 43, 14, 12,
       13, 18], dtype=int64)

In [12]:
df3[df3.bhk>15]

,location,size,total_sqft,bath,price,bhk
1718,2Electronic City Phase II,27 BHK,8000,27.0,230.0,27
3379,1Hanuman Nagar,19 BHK,2000,16.0,490.0,19
3609,Koramangala Industrial Layout,16 BHK,10000,16.0,550.0,16
4684,Munnekollal,43 Bedroom,2400,40.0,660.0,43
11559,1Kasavanhalli,18 Bedroom,1200,18.0,200.0,18


In [13]:
#noticed possible errors on the total_sqft feature eg 43 bedroom with 2400 sqft
df3.total_sqft.unique()

array(['1056', '2600', '1440', ..., '1133 - 1384', '774', '4689'],
      dtype=object)

In [14]:
#more exploration on the total_sqft feature
def is_float(x):
    try:
        float(x)
    except:
        return False
    return True
    

In [18]:
df3[~df3['total_sqft'].apply(is_float)].head(15)
#there are string values in the total_sqft feature

,location,size,total_sqft,bath,price,bhk
30,Yelahanka,4 BHK,2100 - 2850,4.0,186.000,4
122,Hebbal,4 BHK,3067 - 8156,4.0,477.000,4
137,8th Phase JP Nagar,2 BHK,1042 - 1105,2.0,54.005,2
165,Sarjapur,2 BHK,1145 - 1340,2.0,43.490,2
188,KR Puram,2 BHK,1015 - 1540,2.0,56.800,2
410,Kengeri,1 BHK,34.46Sq. Meter,1.0,18.500,1
549,Hennur Road,2 BHK,1195 - 1440,2.0,63.770,2
648,Arekere,9 Bedroom,4125Perch,9.0,265.000,9
661,Yelahanka,2 BHK,1120 - 1145,2.0,48.130,2
672,Bettahalsoor,4 Bedroom,3090 - 5002,4.0,445.000,4


In [19]:
#function to take the average of the range values in the total_sqft feature
def convert_sqft_to_num(x):
    tokens = x.split('-')
    if len(tokens)== 2:
        return (float(tokens[0])+float(tokens[1]))/2
    try:
        return float(x)
    except:
        return None

In [26]:
#testing the function
convert_sqft_to_num('10541-1068')

5804.5

In [25]:
convert_sqft_to_num('10541')

10541.0

In [28]:
df4 = df3.copy()

In [31]:
df4['total_sqft']=df4['total_sqft'].apply(convert_sqft_to_num)


AttributeError: 'float' object has no attribute 'split'

In [33]:
#preview index 30s total_sqft feature
df4.loc[30]

location      Yelahanka
size              4 BHK
total_sqft       2475.0
bath                4.0
price             186.0
bhk                   4
Name: 30, dtype: object

In [34]:
#make a copy of the dataset
df5 = df4.copy()

In [37]:
#feature engineering adding the price per sqft feature
df5['price_per_sqft']=df5['price']*100/df5['total_sqft']
df5.head(10)

,location,size,total_sqft,bath,price,bhk,price_per_sqft
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2,3.699811
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4,4.615385
2,Uttarahalli,3 BHK,1440.0,2.0,62.00,3,4.305556
3,Lingadheeranahalli,3 BHK,1521.0,3.0,95.00,3,6.245891
4,Kothanur,2 BHK,1200.0,2.0,51.00,2,4.250000
5,Whitefield,2 BHK,1170.0,2.0,38.00,2,3.247863
6,Old Airport Road,4 BHK,2732.0,4.0,204.00,4,7.467057
7,Rajaji Nagar,4 BHK,3300.0,4.0,600.00,4,18.181818
8,Marathahalli,3 BHK,1310.0,3.0,63.25,3,4.828244
9,Gandhi Bazar,6 Bedroom,1020.0,6.0,370.00,6,36.274510


In [38]:
#exploring the location feature
df5.location.unique()

array(['Electronic City Phase II', 'Chikka Tirupathi', 'Uttarahalli', ...,
       '12th cross srinivas nagar banshankari 3rd stage',
       'Havanur extension', 'Abshot Layout'], dtype=object)

In [39]:
len(df5.location.unique())

1304

In [40]:
#get how many datapoints are there for a location
df5.location = df5.location.apply(lambda x: x.strip())

#create location stats
location_stats=df5.groupby('location')['location'].agg('count')
location_stats

location
1 Annasandrapalya                                  1
1 Giri Nagar                                       1
1 Immadihalli                                      1
1 Ramamurthy Nagar                                 1
12th cross srinivas nagar banshankari 3rd stage    1
                                                  ..
t.c palya                                          1
tc.palya                                           4
vinayakanagar                                      1
white field,kadugodi                               1
whitefiled                                         1
Name: location, Length: 1293, dtype: int64

In [42]:
#sort the value functions
location_stats=df5.groupby('location')['location'].agg('count').sort_values(ascending=False)
location_stats

#the Whitefiled location has the most entries in the dataset

location
Whitefield               535
Sarjapur  Road           392
Electronic City          304
Kanakpura Road           266
Thanisandra              236
                        ... 
1 Giri Nagar               1
Kanakapura Road,           1
Kanakapura main  Road      1
Karnataka Shabarimala      1
whitefiled                 1
Name: location, Length: 1293, dtype: int64

In [43]:
#get how many locations have less than 10 data points
len(location_stats[location_stats<=10])

1052

In [45]:
#identify locations with less than 10 entries
location_stats_less_than_10 = location_stats[location_stats<=10]
location_stats_less_than_10

location
Basapura                 10
1st Block Koramangala    10
Gunjur Palya             10
Kalkere                  10
Sector 1 HSR Layout      10
                         ..
1 Giri Nagar              1
Kanakapura Road,          1
Kanakapura main  Road     1
Karnataka Shabarimala     1
whitefiled                1
Name: location, Length: 1052, dtype: int64

In [47]:
len(df5.location.unique())

1293

In [49]:
#Have any location less than 10 data points to be reffered to as others

df5.location = df5.location.apply(lambda x: "other" if x in location_stats_less_than_10 else x)
len(df5.location.unique())

242

In [50]:
#preview the locations have been converted to other
df5.head(10)

,location,size,total_sqft,bath,price,bhk,price_per_sqft
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2,3.699811
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4,4.615385
2,Uttarahalli,3 BHK,1440.0,2.0,62.00,3,4.305556
3,Lingadheeranahalli,3 BHK,1521.0,3.0,95.00,3,6.245891
4,Kothanur,2 BHK,1200.0,2.0,51.00,2,4.250000
5,Whitefield,2 BHK,1170.0,2.0,38.00,2,3.247863
6,Old Airport Road,4 BHK,2732.0,4.0,204.00,4,7.467057
7,Rajaji Nagar,4 BHK,3300.0,4.0,600.00,4,18.181818
8,Marathahalli,3 BHK,1310.0,3.0,63.25,3,4.828244
9,other,6 Bedroom,1020.0,6.0,370.00,6,36.274510
